In [ ]:
# Install openvino package
%pip install -q "openvino>=2023.1.0" nncf

In [ ]:
from pathlib import Path

# Set the data and model directories
DATA_DIR = Path("data")
MODEL_DIR = Path('model')
model_repo = 'pytorch-cifar-models'

DATA_DIR.mkdir(exist_ok=True)
MODEL_DIR.mkdir(exist_ok=True)

In [ ]:
import torch
from pprint import pprint
pprint(torch.hub.list("chenyaofo/pytorch-cifar-models", force_reload=True))

In [ ]:
import sys

if not Path(model_repo).exists():
    !git clone https://github.com/chenyaofo/pytorch-cifar-models.git

sys.path.append(model_repo)

In [ ]:
from pytorch_cifar_models import cifar10_resnet32

model = cifar10_resnet32(pretrained=True)

In [ ]:
import openvino as ov

model.eval()

ov_model = ov.convert_model(model, input=[(1,3,96,96)])

ov.save_model(ov_model, MODEL_DIR / "mobilenet_v2.xml")


In [ ]:
import torch
from torchvision import transforms
from torchvision.datasets import STL10

DATA_DIR = './data'  # Define your data directory here

# Load STL10 dataset without normalization for computing mean and std
dataset_for_stats = STL10(root=DATA_DIR, split='train', transform=transforms.ToTensor(), download=True)

data_loader_for_stats = torch.utils.data.DataLoader(dataset_for_stats, batch_size=1000, shuffle=False, num_workers=4)

mean = 0.0
for images, _ in data_loader_for_stats:
    batch_samples = images.size(0)  # batch size
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
mean = mean / len(data_loader_for_stats.dataset)

variance = 0.0
for images, _ in data_loader_for_stats:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    variance += ((images - mean.unsqueeze(1))**2).sum([0, 2])
std = torch.sqrt(variance / (len(data_loader_for_stats.dataset) * 96 * 96))

# Now, use the computed mean and std values for normalization and set up the dataset and loader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((mean[0], mean[1], mean[2]), (std[0], std[1], std[2]))
])
dataset = STL10(root=DATA_DIR, split='test', transform=transform, download=True)

val_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
)


In [ ]:
import nncf

def transform_fn(data_item):
    image_tensor = data_item[0]
    return image_tensor.numpy()

quantization_dataset = nncf.Dataset(val_loader, transform_fn)

In [ ]:
quant_ov_model = nncf.quantize(ov_model, quantization_dataset)

In [ ]:
ov.save_model(quant_ov_model, MODEL_DIR / "quantized_mobilenet_v2.xml")

In [ ]:
from tqdm.notebook import tqdm
import numpy as np

def test_accuracy(ov_model, data_loader):
    correct = 0
    total = 0
    for (batch_imgs, batch_labels) in tqdm(data_loader):
        result = ov_model(batch_imgs)[0]
        top_label = np.argmax(result)
        correct += top_label == batch_labels.numpy()
        total += 1
    return correct / total

In [ ]:
import ipywidgets as widgets

core = ov.Core()
device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

device

In [ ]:
core = ov.Core()
compiled_model = core.compile_model(ov_model, device.value)
optimized_compiled_model = core.compile_model(quant_ov_model, device.value)

orig_accuracy = test_accuracy(compiled_model, val_loader)
optimized_accuracy = test_accuracy(optimized_compiled_model, val_loader)

In [ ]:
# Inference FP16 model (OpenVINO IR)
!benchmark_app -m "model/mobilenet_v2.xml" -d $device.value -api async -t 15

In [ ]:
# Inference INT8 model (OpenVINO IR)
!benchmark_app -m "model/quantized_mobilenet_v2.xml" -d $device.value -api async -t 15

In [ ]:
import torch
import numpy as np
from torchvision.datasets import STL10
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Define all possible labels from the STL10 dataset
labels_names = ["airplane", "bird", "car", "cat", "deer", "dog", "horse", "monkey", "ship", "truck"]

DATA_DIR = './data'

# Load the STL10 dataset and compute mean and std for normalization (assuming you've done this before)
# ...

# Create the dataset and data loader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((mean[0], mean[1], mean[2]), (std[0], std[1], std[2]))
])
dataset = STL10(root=DATA_DIR, split='test', transform=transform, download=True)
val_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)

all_pictures, all_labels = [], []
for img, lbl in val_loader:
    all_pictures.append(img.squeeze().numpy())
    all_labels.append(lbl.item())

def plot_pictures(indexes: list):
    images, label_texts = [], []
    for idx in indexes:
        assert idx < len(all_pictures), f'Cannot get index {idx}, there are only {len(all_pictures)} images'
        pic = np.rollaxis(all_pictures[idx], 0, 3)
        images.append(pic)
        label_texts.append(labels_names[all_labels[idx]])

    f, axarr = plt.subplots(1, 4)
    for i, ax in enumerate(axarr):
        ax.imshow(images[i])
        ax.set_title(label_texts[i])
    plt.show()

def infer_on_pictures(model, indexes: list, all_pictures):
    predicted_labels = []
    for idx in indexes:
        assert idx < len(all_pictures), f'Cannot get index {idx}, there are only {len(all_pictures)} images'
        with torch.no_grad():
            outputs = model(torch.tensor(all_pictures[idx]).unsqueeze(0))

            # Directly extract the tensor (value) from the OVDict without relying on the key
            result_np = list(outputs.values())[0]  # get the first value from the dictionary

            # Convert the numpy array to a PyTorch tensor
            result = torch.tensor(result_np)

            predicted_label = labels_names[torch.argmax(result)]
            predicted_labels.append(predicted_label)
    return predicted_labels



# Assuming you've defined and loaded `compiled_model` and `optimized_compiled_model`
indexes_to_infer = [7, 12, 15, 20]  # To plot, specify 4 indexes.

plot_pictures(indexes_to_infer)

results_float = infer_on_pictures(compiled_model, indexes_to_infer, all_pictures)
results_quanized = infer_on_pictures(optimized_compiled_model, indexes_to_infer, all_pictures)

print(f"Labels for picture from float model : {results_float}.")
print(f"Labels for picture from quantized model : {results_quanized}.")

